<h1 align="center">
	Variational quantum eigensolver
</h1>

Wie sie in der Vorlesung gelernt haben, ist der Variational quantum eigensolver (VQE) ein hybrider quanten-klassischer Algorithmus, welcher den kleinsten Eigenwert eines gegebenen Hamiltonians findet. Dieser kann genutzt werden, um den Grundzustand eines Moleküls zu finden. 

Die Idee des VQE-Algorithmus ist es, aus einem gegebenen Hamiltonian mit gegebenem $\left| \psi \right\rangle$ den Erwartungswert zu messen. Hier ein Beispiel für einen 2-Qubit-Hamiltonian:

$$H = a \cdot II + b \cdot IZ + c \cdot ZI + d \cdot ZZ + e\cdot XX.$$

$$\left\langle H \right\rangle = \left\langle \psi \right| H \left| \psi \right\rangle = a \cdot \left\langle \psi \right| II \left| \psi \right\rangle + b \cdot \left\langle \psi \right| IZ \left| \psi \right\rangle + c \cdot \left\langle \psi \right| ZI \left| \psi \right\rangle+ d \cdot \left\langle \psi \right| ZZ \left| \psi \right\rangle+ e \cdot \left\langle \psi \right| XX \left| \psi \right\rangle.$$

Der Algorithmus konstruiert für jeden Pauli-Term einen Quantenschaltkreis und berechnet den Erwartungswert des entsprechenden Terms. Durch Addition der einzelnen Terme erhält man die Erwartungswert von $H$.

Der Eigenvektor $\left| \psi_g \right\rangle$ der den Erwartungswert $\left\langle H \right\rangle$ minimiert, ist der Eigenvektor von $H$ zum kleinsten Eigenwert. In dem Algorithmus werden die Versuchszustände aus einer parametrisierten Schaltung erzeugt. Durch Ändern der Parameter erhält man verschiedene Ansatzzustände. Wenn die Schaltung mit ihren Parametern gut genug ist, erhält man eine Lösung. 

Die Optimierung der Parameter wird von einem klassischen Computer gesteuert. Bei jedem Schritt ändert der klassische Computer die Parameter mithilfe einer Optimierungsmethode, um einen Ansatzzustand zu erstellen, der einen kleineren Erwartungswert aufweist als frühere Ansatzzustände. Auf diese Weise arbeiten der klassische Computer und der Quantencomputer zusammen, um das Ziel des Algorithmus zu erreichen (die Grundzustandsenergie zu finden). Daher ist VQE ein quanten-klassischer Hybridalgorithmus.

In [ ]:
import numpy as np
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver
from qiskit.aqua.components.optimizers import SPSA

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

Wir verwenden den in der Vorlesung gezeigten "Ry" - Ansatz für 2 Qubits, d.h. erst auf beide Qubits jeweils ein Ry - Gatter anwenden, dann CNOT, dann wieder Ry, usw. (insgesamt $k$ Iterationen). Die Anzahl der Parameter beträgt also $2*(k+1)$

In [ ]:
def quantum_state_prep(k, angles):  #angles sollte eine Liste der Länge 2*(k+1)
    #sein
    circ=QuantumCircuit(2,2)
#### Schreiben Sie ihren Code hier #########
    circ.ry(angles[0],0)
    circ.ry(angles[1],1)
    for i in range(k):
        circ.cx(0,1)
        circ.ry(angles[2*i+2],0)
        circ.ry(angles[2*i+3],1)

    return circ 

In [ ]:
# Testen Sie die Ausgabe hier:
qc = quantum_state_prep(1, [np.pi/8, np.pi/6,np.pi/4,np.pi/2])
qc.draw()

Um den Erwartungswert von $H$ zu bestimmen, benötigen Sie Messungen in den Basen $ZZ$ und $XX$. Die folgenden Funktionen sollen den gegebenen Schaltkreis erst kopieren (mit der Funktion .copy(), damit er nicht überschrieben wird), und dann die nötigen Messungen hinzufügen:

In [ ]:
def measure_zz_circuit(given_circuit):
    ### Schreiben Sie ihren Code hier ######
    qc=given_circuit.copy()
    qc.measure([0,1],[0,1])
    
    return qc
    
    ##### Ende #####
def measure_xx_circuit(given_circuit):
    ### Schreiben Sie ihren Code hier ######
    qc=given_circuit.copy()
    qc.h([0,1])
    qc.measure([0,1],[0,1])
    
    return qc
    ##### Ende #####

In [ ]:
#Testen sie ihren Schaltkreis
xx_meas = measure_xx_circuit(qc)
xx_meas.draw()

Für die folgenden Funktionen sollen Sie die Erwartungswerte auswerten und den Mittelwert von $\langle ZZ \rangle$, $\langle ZI \rangle$, $\langle IZ \rangle$ und $\langle XX \rangle$ ermitteln. Dazu müssen Sie in der Funktion die Messung durchführen, die Resultate und 'Counts' extrahieren. 
Für die unterschiedlichen Anteile müssen Sie sich überlegen, welche 'Counts' gezählt werden sollen.

In [ ]:
def measure_zz(given_circuit, backend, shots):
    ### Schreiben Sie ihren Code hier ######
    qc=measure_zz_circuit(given_circuit)
    job = execute(qc,backend=backend,shots=shots)
    result=job.result()
    counts=result.get_counts(given_circuit)
    
    iz=0
    zi=0
    zz=0
    for c in counts:
        if c=='00':
            iz+=counts[c]/shots
            zi+=counts[c]/shots
            zz+=counts[c]/shots
        if c=='01':
            iz+=counts[c]/shots
            zi-=counts[c]/shots
            zz-=counts[c]/shots
        if c=='10':
            iz-=counts[c]/shots
            zi+=counts[c]/shots
            zz-=counts[c]/shots
        if c=='11':
            iz-=counts[c]/shots
            zi-=counts[c]/shots
            zz+=counts[c]/shots
    return iz, zi, zz
def measure_xx(given_circuit, backend, shots):
    ### Schreiben Sie ihren Code hier ######
    qc=measure_xx_circuit(given_circuit)
    job = execute(qc,backend=backend,shots=shots)
    result=job.result()
    counts=result.get_counts(given_circuit)
    
    xx=0
    for c in counts:
        if c=='00':
            xx+=counts[c]/shots
        if c=='01':
            xx-=counts[c]/shots
        if c=='10':
            xx-=counts[c]/shots
        if c=='11':
            xx+=counts[c]/shots
    return xx

In [ ]:
#Sie können hier ihre Funktionen testen
backend = Aer.get_backend('qasm_simulator')
iz, zi, zz = measure_zz(qc, backend, 1000)
print("<IZ> =", str(iz)," <ZI> =", str(zi)," <ZZ> =", str(zz))

Nachdem Sie die einzelnen Erwartungswerte ermittelt haben, benötigen Sie eine Funktion ```get_energy```um die gesamte Energie des Systems zu berechnen. Diese Funktion wird später vom SPSA-Optimierer aufgerufen und sollte daher nur die Liste der Parameter des Schaltkreises als Argument enthalten.

In [ ]:
def get_energy(angles):
    ### Schreiben Sie ihren Code hier ####
    qc = quantum_state_prep(1, angles)
    shots=1024
    backend = Aer.get_backend('qasm_simulator')
    
    iz, zi, zz = measure_zz(qc, backend, shots)
    xx = measure_xx(qc, backend, shots)
    
    #### Ende ####
    # Hier finden Sie die Pauli Operatoren von H2
    
    energy = (-1.0523732)*1 + (0.39793742)*iz + (-0.3979374)*zi + (-0.0112801)*zz + (0.18093119)*xx
    
    return energy

Nachfolgend die exakte Berechnung des kleinsten Eigenwerts

In [ ]:
def exact_solver():
    def hamiltonian_operator(a, b, c, d, e):
        pauli_dict = {
            'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "II"},
                       {"coeff": {"imag": 0.0, "real": b}, "label": "IZ"},
                       {"coeff": {"imag": 0.0, "real": c}, "label": "ZI"},
                       {"coeff": {"imag": 0.0, "real": d}, "label": "ZZ"},
                       {"coeff": {"imag": 0.0, "real": e}, "label": "XX"}
                       ]
        }
        return WeightedPauliOperator.from_dict(pauli_dict)
    a, b, c, d, e = ((-1.0523732), (0.39793742), (-0.3979374), (-0.0112801),(0.18093119))
    H = hamiltonian_operator(a, b, c, d, e)
    exact_result = NumPyEigensolver(H).run()
    reference_energy = min(np.real(exact_result.eigenvalues))
    return reference_energy

Nun wählen wir willkürlich irgendwelche Startwerte für die zu optimierenden Parameter und vergleichen das Ergebnis mit der exakten Lösung:

In [ ]:
exact_energy = exact_solver()
angles = [np.pi/8, np.pi/6,np.pi/4,np.pi/2] #Startwerte
energy = get_energy(angles)
print('Die exakte Grundzustandsenergie ist: {}'.format(exact_energy))
print('Die abgeschätzte Grundzustandsenergie für die Parameter ',angles,' ist: {}'.format(energy))

Wie sie sehen können ist das Ergebnis noch nicht minimal, daher müssen Sie noch eine Funktion schreiben, welche die Winkel in der Funktion ```quantum_state_prep``` so variiert, dass eine minimale Energie abgeschätzt wird. Hierfür können Sie den SPSA-Algorithmus verwenden. Zur Erklärung, wie man diesen einbindet, hier die Minimierung der Funktion
$$f(a_0,a_1)=a_0^2+2 a_1^2-2a_0-a_1$$
als kleines Beispiel:

In [ ]:
def f(a):
    return a[0]**2+2*a[1]**2-2*a[0]-a[1]

optimizer=SPSA(maxiter=100) #Größeres maxiter führt normalerweise zu besseren
#Ergebnissen, dauert aber länger
result=optimizer.optimize(2,f,initial_point=[2,1]) #erstes Argument: Zahl der
# Parameter, zweites Argument: Funktion, drittes Argument
#initial_point = Startwert
print("Optimale Parameter:",result[0])
print("Minimaler Funktionswert:",result[1])

Führen Sie nun den VQE-Algorithmus inklusive Minimierung durch. Kann man mit einer Iteration ($k=1$) schon das exakte Ergebnis erreichen?

In [ ]:
#### Schreiben Sie ihren Code hier:
optimizer=SPSA(maxiter=50)
result=optimizer.optimize(4,get_energy,initial_point=[0,0,0,0])
print(result)

Sie haben nun gesehen, wie der VQE Algorithmus programmiert werden kann und damit die Grundzustandsenergie von $H_2$ berechnet. Um zu sehen, wie man die vorgefertigten Qiskit-Module hierfür benutzen kann, hier noch ein Beispiel für $LiH$ (4 Qubits). Man braucht hierfür das Quantenchemie-Modul pyscf (wenn Sie es nicht installiert haben, können Sie das Programm auf IBM Quantum Experience laufen lassen.) Die folgenden Zellen müssen Sie nur auswerten.

In [ ]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
 def get_qubit_op(dist): #Berechnung des Hamiltonians 
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift

In [ ]:
backend = BasicAer.get_backend("statevector_simulator")
#Der statevector Simulator führt im Gegensatz zum qasm simulator keine Messungen
#durch, sondern berechnet den Quantenzustand vollständig, so dass man Erwartungswerte
#ohne statistische Schwankungen wegen endlicher Zahl von Messungen berechnen kann.
#Dies geht nur mit dem simulator (nicht auf einem echten backend)
distances = np.arange(0.5, 3.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5) #### kann abgeändert werden, andere Optimizer: COBYLA, L_BFGS, SPSA
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD( # eine "coupled-cluster" variational Form aus der Quantenchemie
        num_orbitals=num_spin_orbitals, 
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")

Das gleiche mit der Ry-Form aus der Vorlesung

In [ ]:
from qiskit.circuit.library import TwoLocal
vqe_energies2 = []
optimizer2 = SLSQP(maxiter=100) #### kann abgeändert werden, andere Optimizer: COBYLA, L_BFGS, SPSA
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    #result = NumPyEigensolver(qubitOp).run()
    #exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form2=EfficientSU2(qubitOp.num_qubits, su2_gates=['ry'], entanglement='linear', reps=1)
    vqe2 = VQE(qubitOp, var_form2, optimizer2)
    vqe_result2 = np.real(vqe2.run(backend)['eigenvalue'] + shift)
    vqe_energies2.append(vqe_result2)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result2)

In [ ]:
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE UCCSD Energy")
plt.plot(distances, vqe_energies2, label="VQE RY Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
plt.show()

Beim Vergleich ist zu beachten, dass die Ry-Form wesentlich kürzer ist:

In [ ]:
print(var_form2)

Im Gegensatz zu UCCSD:

In [ ]:
qc=var_form.construct_circuit([1,2,3,4,5,6,7,8])
qc.decompose().draw('mpl')